In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution

warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
def free_mem():
    """Try to return freed memory back to the OS (no-op on some platforms)."""
    try:
        libc = ctypes.CDLL(None)
        if hasattr(libc, "malloc_trim"):
            libc.malloc_trim(0)
    except Exception:
        pass

In [3]:
# --- DUCKDB CHUNK GENERATOR (Fixed) ---

def duckdb_chunk_generator(conn, sql_query, chunk_size):
    """Generates Pandas DataFrames in chunks directly from DuckDB cursor."""
    cursor = conn.cursor() 
    cursor.execute(sql_query)
    
    while True:
        # Uses the corrected method name: fetch_df_chunk
        chunk = cursor.fetch_df_chunk(chunk_size) 
        if chunk is None or chunk.empty:
            break
        yield chunk

# --- ROBUST FILE EXTRACTION (Fixed from OSErrors) ---

def extract_files_to_disk(zip_path, output_dir):
    """Cleans directory and extracts all Parquet files from ZIP."""
    if output_dir.exists():
        logging.info(f"🧹 Clearing existing directory: {output_dir.resolve()}")
        # Robust cleanup to avoid OS/lock issues
        try:
            shutil.rmtree(output_dir)
        except OSError:
             for item in output_dir.iterdir():
                if item.is_dir():
                    shutil.rmtree(item)
                else:
                    os.remove(item) 
             os.rmdir(output_dir)

    output_dir.mkdir(parents=True)
        
    logging.info("🔄 Extracting ALL Parquet files from ZIP to disk...")
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            all_files_to_extract = [f for f in z.namelist() if f.endswith(".parquet")]
            logging.info(f"🔎 Found {len(all_files_to_extract)} total Parquet files in archive.")
            for filename in all_files_to_extract:
                z.extract(filename, path=output_dir)
            return len(all_files_to_extract)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ ZIP file not found at: {zip_path}") from None

def setup_duckdb_query(output_dir, utc_start, utc_end, core_cols):
    """Sets up DuckDB connection and SQL query."""
    parquet_glob_path = str(output_dir.joinpath("**/*.parquet"))
    # Only select the columns you need for Stage 1 processing
    column_list = ", ".join([f'"{c}"' for c in core_cols])
    
    # CRITICAL: Predicate Pushdown filter on the internal 'timestamp' column
    sql_query = f"""
        SELECT {column_list}
        FROM read_parquet('{parquet_glob_path}')
        WHERE 
            "timestamp" >= '{utc_start.isoformat()}' AND 
            "timestamp" < '{utc_end.isoformat()}'
    """
    return duckdb.connect(), sql_query

In [4]:
def read_parquet_subset(parquet_path: str, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
    """
    Reads a subset of the processed Feather file using a date filter
    applied directly by DuckDB (predicate pushdown).
    
    Args:
        feather_path: Path to the processed Feather file.
        start_dt: Start datetime for the filter (inclusive).
        end_dt: End datetime for the filter (exclusive).
        
    Returns:
        A new DataFrame containing only the filtered data.
    """
    logging.info(f"Loading data subset from {start_dt} to {end_dt}...")
    
    # Use DuckDB to query the Feather file directly on disk
    con = duckdb.connect()
    
    # The SQL query filters rows on the disk file based on the 'timestamp' column.
    sql_query = f"""
        SELECT *
        FROM read_parquet('{parquet_path}')
        WHERE 
            "timestamp" >= '{start_dt.isoformat()}' AND 
            "timestamp" < '{end_dt.isoformat()}'
    """
    
    # Fetch the filtered, smaller DataFrame
    df_subset = con.execute(sql_query).fetchdf()
    con.close()
    
    logging.info(f"✅ Loaded {len(df_subset):,} rows for the requested subset.")
    return df_subset

In [5]:
# --- FIXED CATEGORY DEFINITIONS ---
# These fixed labels ensure your output columns are consistent across all data chunks.
FIXED_CATEGORIES = {
    'maxtemp_bucket': ["<28", "28–32", "32–35", "35–40", ">40"],
    'temp_delta_bucket': ["<2", "2–5", "5–8", ">8"],
    'volt_delta_bucket': ["0–10", "10–20", "20–30", ">30"] 
}
# ----------------------------------

def create_session_report(
    df: pd.DataFrame, 
    timezone: str = 'Asia/Kolkata',
    min_discharging_duration_min: float = 1.0
) -> pd.DataFrame:
    """
    Performs sessionization and calculates the percentage distribution of 
    key operational buckets per session using a fixed set of categories.
    """
    # 1. CRITICAL STEP: Ensure data is sorted by ID and Timestamp
    df = df.sort_values(['id', 'timestamp']).reset_index(drop=True)
    
    # Ensure 'timestamp' is timezone-aware IST
    if not isinstance(df['timestamp'].dtype, pd.DatetimeTZDtype):
        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True).dt.tz_convert(timezone)
        except Exception:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # 2. Identify the start of a new session
    df['id_change'] = df['id'].ne(df['id'].shift())
    df['mode_change'] = df['alt_mode'].ne(df['alt_mode'].shift())
    df['session_start'] = df['id_change'] | df['mode_change']
    df['unique_session_id'] = df['session_start'].cumsum()

    # 3. Aggregate Session Metrics (Non-bucket columns)
    session_df = df.groupby('unique_session_id').agg(
        id=('id', 'first'),
        reg_num=('reg_num', 'first'),
        customer=('customer', 'first'),
        model=('model', 'first'),
        # mode=('mode', 'first'),
        alt_mode=('alt_mode', 'first'),
        start_time=('timestamp', 'min'),
        end_time=('timestamp', 'max'),
    ) # Unique_session_id is the index
    
    # 4. --- NEW: Calculate Percentage of Buckets per Session using FIXED CATEGORIES ---
    bucket_cols = ['maxtemp_bucket', 'temp_delta_bucket', 'volt_delta_bucket']
    agg_dfs = []
    
    for col in bucket_cols:
        if col in df.columns and col in FIXED_CATEGORIES:
            
            # --- Key Fix: Convert to CategoricalDtype with fixed categories ---
            # This ensures all categories are present, even those with zero count.
            df[col] = pd.Categorical(df[col], categories=FIXED_CATEGORIES[col], ordered=True)
            # -------------------------------------------------------------------
            
            # Calculate value counts normalized to get percentages
            pct_series = round(df.groupby('unique_session_id')[col].value_counts(normalize=True) * 100,2)
            
            # Unstack the bucket labels to create new columns, fill missing (zero) categories
            pivot_df = pct_series.unstack(fill_value=0)
            
            # Rename columns for clear identification and safe use (e.g., replaces '–' with '_')
            pivot_df.columns = [
                f"{col}_{str(c).replace('–', '_').replace('<', 'lt').replace('>', 'gt')}_pct" 
                for c in pivot_df.columns
            ]
            agg_dfs.append(pivot_df)

    # Merge all percentage tables back into the main session_df
    if agg_dfs:
        for agg_df in agg_dfs:
            session_df = session_df.join(agg_df, how='left')
    
    session_df = session_df.reset_index(drop=True) # Now reset index

    # 5. Final Calculations and Formatting
    
    # Calculate Duration in MINUTES (seconds / 60) and round to 2 decimal places
    session_df['duration'] = (
        (session_df['end_time'] - session_df['start_time']).dt.total_seconds() / 60
    ).round(2)

    # Add sequential session number (e.g., 1, 2, 3...) per vehicle ID
    session_df['session'] = session_df.groupby('id').cumcount() + 1
    
    # 6. Final Output Selection and Formatting
    
    session_report = session_df.copy()

    # Define the core columns to ensure they are first
    core_cols = ['id', 'reg_num', 'customer', 'model', 'alt_mode', 'session', 'start_time', 'end_time', 'duration']
    
    # Order columns: core columns first, then new percentage columns
    new_cols = [c for c in session_report.columns if c not in core_cols]
    session_report = session_report[core_cols + new_cols]
    
    # --- FINAL FORMATTING ---
    # Round times to seconds precision (floor)
    session_report['start_time'] = session_report['start_time'].dt.floor('S')
    session_report['end_time'] = session_report['end_time'].dt.floor('S')
    session_report['date_val'] = session_report['start_time'].dt.date

    # Drop Timezone information (+05:30)
    session_report['start_time'] = session_report['start_time'].dt.tz_localize(None)
    session_report['end_time'] = session_report['end_time'].dt.tz_localize(None)
    # -------------------------

    return session_report

In [6]:
from datetime import datetime, timedelta
import pandas as pd
import gc
import logging # Required for managing log levels
from tqdm.auto import tqdm 
import time # For timing the execution

# NOTE: The create_session_report and read_parquet_subset functions 
# must be defined/imported before running this code.

def run_multi_day_session_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
) -> pd.DataFrame:
    """
    Iteratively loads, processes, and aggregates session reports for a defined number of days,
    displaying a progress bar and suppressing INFO logging during the process.
    """
    all_reports = []
    total_rows_ingested = 0 # <--- ADD THIS LINE HERE

    # --- Logging Suppression: Temporarily set log level to WARNING to hide INFO messages ---
    current_log_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING) 
    
    try:
        progress_bar = tqdm(range(num_days), desc="Processing Daily Sessions", unit="day", colour="cyan") # Keeps the cyan color you had, distinct from the other function)
        
        for i in progress_bar:
            current_start_dt = start_date + timedelta(days=i)
            current_end_dt = current_start_dt + timedelta(days=1)
            
            progress_bar.set_postfix_str(f"Date: {current_start_dt.strftime('%Y-%m-%d')}")
            
            # 1. Load the small subset (Logging is suppressed here)
            df_subset = read_parquet_subset(
                parquet_path=parquet_path,
                start_dt=current_start_dt,
                end_dt=current_end_dt
            )
            
            if df_subset.empty:
                del df_subset
                gc.collect()
                continue
            
            total_rows_ingested += len(df_subset)   #count rows being processed

            # 2. Process the subset
            session_report_chunk = create_session_report(df_subset)
            
            # 3. Aggregate the result
            all_reports.append(session_report_chunk)
            
            # 4. Explicit Memory Management
            del df_subset 
            gc.collect()  
            
    finally:
        # --- Restore Logging Level ---
        logging.getLogger().setLevel(current_log_level)
        
    # 5. Final Concatenation
    if all_reports:
        final_report_df = pd.concat(all_reports, ignore_index=True)
        return final_report_df, total_rows_ingested
    else:
        return pd.DataFrame(), total_rows_ingested

# =========================================================================
# --- Execution Example ---
# =========================================================================

# Define the 75-day process window
filter_start_date = datetime(2025, 9, 1) # Start date
total_days_to_process = 75

print(f"--- Starting 75-Day Session Analysis ---")
print(f"Processing range: {filter_start_date.strftime('%Y-%m-%d')} to {(filter_start_date + timedelta(days=total_days_to_process-1)).strftime('%Y-%m-%d')}")
print("-" * 40)

start_time = time.time()

# Run the iterative analysis
session_report_df, total_rows = run_multi_day_session_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=filter_start_date,
    num_days=total_days_to_process
)

end_time = time.time()
elapsed_time_sec = end_time - start_time
elapsed_time_min = elapsed_time_sec / 60

# --- Final Summary ---
print("\n" + "=" * 40)
print("✅ ANALYSIS COMPLETE")
print(f"Total time taken: {elapsed_time_sec:.2f} seconds ({elapsed_time_min:.2f} minutes)")
print(f"Days processed:   {total_days_to_process}")
print(f"Total Rows Ingested: {total_rows:,}")
print(f"Final Report Shape: {session_report_df.shape} (Rows: {session_report_df.shape[0]}, Columns: {session_report_df.shape[1]})")
print("=" * 40)

--- Starting 75-Day Session Analysis ---
Processing range: 2025-09-01 to 2025-11-14
----------------------------------------


Processing Daily Sessions: 100%|██████████| 75/75 [02:29<00:00,  2.00s/day, Date: 2025-11-14]



✅ ANALYSIS COMPLETE
Total time taken: 151.01 seconds (2.52 minutes)
Days processed:   75
Total Rows Ingested: 51,788,694
Final Report Shape: (5147217, 23) (Rows: 5147217, Columns: 23)


In [8]:
session_report_df.to_csv('bcs_analysis_sessions.csv')
session_report_df.head(5)
# session_report_df[(session_report_df.id == '11')&(session_report_df.date_val=='2025-09-08')&(session_report_df.alt_mode == 'CHARGING')]

KeyboardInterrupt: 

In [ ]:
import pandas as pd

def weighted_avg_factory(df, weight_col='duration'):
    """
    Creates a callable function for pandas aggregation that calculates 
    the weighted average of a series using a specific weight column from the 
    original DataFrame (df) based on the group's indices.
    """
    def weighted_avg(series):
        # Retrieve the original 'duration' (weights) for the rows in the current group (series)
        weights = df.loc[series.index, weight_col]
        
        # Guard against division by zero (e.g., if total duration is 0)
        if weights.sum() == 0:
            return 0.0
            
        # Weighted average calculation
        return ((series * weights).sum() / weights.sum()).round(4)

    return weighted_avg

# --- ASSUMING session_report_df is your input DataFrame (the 75-day report) ---
# NOTE: Replace 'session_report_df' with the actual variable name from your script (e.g., the output of run_multi_day_session_analysis)
df = session_report_df 

# Ensure 'duration' is numeric and set up the weighted average function
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
w_avg = weighted_avg_factory(df, 'duration')

# Define all percentage columns for aggregation
pct_cols = [col for col in df.columns if col.endswith('_pct')]

# Define the full aggregation dictionary
agg_dict = {
    'reg_num': 'first',
    'customer': 'first',
    'model': 'first',
    'session': 'count',  # Becomes total_sessions
    'duration': 'sum'   # Becomes total_duration
}

# Add weighted average for all percentage columns
weighted_agg_dict = {col: w_avg for col in pct_cols}
agg_dict.update(weighted_agg_dict)

In [ ]:
# 1. Perform aggregation grouped by 'id' and 'mode'
report_agg = df.groupby(['id', 'alt_mode', 'date_val']).agg(agg_dict).reset_index()

# 2. Final Formatting and Column Renaming/Reordering
report_agg = report_agg.rename(columns={'session': 'total_sessions', 'duration': 'total_duration'})
report_agg['total_duration'] = report_agg['total_duration'].round(2)

report_agg['maxtemp_bucket_lt28_pct'] = report_agg['maxtemp_bucket_lt28_pct'].round(2)
report_agg['maxtemp_bucket_28_32_pct'] = report_agg['maxtemp_bucket_28_32_pct'].round(2)
report_agg['maxtemp_bucket_32_35_pct'] = report_agg['maxtemp_bucket_32_35_pct'].round(2)
report_agg['maxtemp_bucket_35_40_pct'] = report_agg['maxtemp_bucket_35_40_pct'].round(2)
report_agg['maxtemp_bucket_gt40_pct'] = report_agg['maxtemp_bucket_gt40_pct'].round(2)

report_agg['temp_delta_bucket_lt2_pct'] = report_agg['temp_delta_bucket_lt2_pct'].round(2)
report_agg['temp_delta_bucket_2_5_pct'] = report_agg['temp_delta_bucket_2_5_pct'].round(2)
report_agg['temp_delta_bucket_5_8_pct'] = report_agg['temp_delta_bucket_5_8_pct'].round(2)
report_agg['temp_delta_bucket_gt8_pct'] = report_agg['temp_delta_bucket_gt8_pct'].round(2)

report_agg['volt_delta_bucket_0_10_pct'] = report_agg['volt_delta_bucket_0_10_pct'].round(2)
report_agg['volt_delta_bucket_10_20_pct'] = report_agg['volt_delta_bucket_10_20_pct'].round(2)
report_agg['volt_delta_bucket_20_30_pct'] = report_agg['volt_delta_bucket_20_30_pct'].round(2)
report_agg['volt_delta_bucket_gt30_pct'] = report_agg['volt_delta_bucket_gt30_pct'].round(2)


# Reorder columns as requested
final_columns = [
    'id', 'reg_num', 'customer', 'model', 'alt_mode', 'date_val','total_sessions', 'total_duration'
] + pct_cols

report_agg = report_agg[final_columns]


report_agg.to_csv('bcs_analysis_report_v1.csv')
report_agg.head()

report_agg[(report_agg.maxtemp_bucket_35_40_pct>40)|(report_agg.maxtemp_bucket_gt40_pct>40)].to_csv('bcs_session_maxT.csv')
report_agg[(report_agg.maxtemp_bucket_35_40_pct>40)|(report_agg.maxtemp_bucket_gt40_pct>40)].head()
report_agg[(report_agg.temp_delta_bucket_5_8_pct>40)|(report_agg.temp_delta_bucket_gt8_pct>40)].to_csv('bcs_session_deltaT.csv')
report_agg[(report_agg.temp_delta_bucket_5_8_pct>40)|(report_agg.temp_delta_bucket_gt8_pct>40)].head()
report_agg[(report_agg.volt_delta_bucket_20_30_pct>40)|(report_agg.volt_delta_bucket_gt30_pct>40)].to_csv('bcs_session_deltaV.csv')
report_agg[(report_agg.volt_delta_bucket_20_30_pct>40)|(report_agg.volt_delta_bucket_gt30_pct>40)].head()

In [ ]:
import pandas as pd
import numpy as np

def compute_detailed_hotspot(df, item_id_col='tc_id', pack_id_col='pack_id'):
    """
    Computes detailed hotspot analysis for a single vehicle on a single day.
    
    Args:
        df (pd.DataFrame): Input DataFrame for a single vehicle and day.
        item_id_col (str): The column representing the individual item ID 
                           (e.g., 'tc_id' or 'pack_id').
        pack_id_col (str): The column representing the pack ID (e.g., 'pack_id').
                           This is separated for TC-level grouping.
    
    Returns:
        pd.DataFrame: Detailed hotspot metrics grouped by item_id and pack_id.
    """
    if df.empty:
        return pd.DataFrame()

    # Basic Info
    date_val = df['date_val'].iloc[0]
    vid = df['id'].iloc[0]
    
    # Pre-calculate total time for the day (in seconds)
    total_day_sec = df['dt_sec'].sum()
    # total_day_mins = min(total_day_sec / 60.0, 1440.0) # Clipping the daily duration
    total_day_mins = total_day_sec / 60.0
    
    # --- Helper to aggregate ---
    def get_agg(subset_df, id_col, val_col, pack_col):
        if subset_df.empty:
            return pd.Series(dtype='float64')
        # Group by the item ID and its associated Pack ID
        return subset_df.groupby([id_col, pack_col])[val_col].sum()

    # Determine the columns to group by (Max TC/Pack and Min TC/Pack)
    # The columns in the raw data are fixed: batt_maxtemp_tc, pack_id_max, etc.
    group_cols = [item_id_col, pack_id_col]

    # Use the relevant columns from the raw data for aggregation:
    c_df = df[df['alt_mode'] == 'CHARGING']
    dc_df = df[df['alt_mode'] == 'DISCHARGING']
    
    # 1. C-Max (Charging, Max Temp)
    s_cmax = get_agg(c_df, 'batt_maxtemp_tc', 'dt_sec', 'pack_id_max')
    
    # 2. DC-Max (Discharging, Max Temp)
    s_dcmax = get_agg(dc_df, 'batt_maxtemp_tc', 'dt_sec', 'pack_id_max')
    
    # 3. C-Min (Charging, Min Temp)
    s_cmin = get_agg(c_df, 'batt_mintemp_tc', 'dt_sec', 'pack_id_min')
    
    # 4. DC-Min (Discharging, Min Temp)
    s_dcmin = get_agg(dc_df, 'batt_mintemp_tc', 'dt_sec', 'pack_id_min')
    
    # --- Merge all series ---
    dfs = []
    
    # Combine the index names for consistent merging (TC ID and Pack ID)
    index_names = ['tc_id', 'pack_id'] 

    if not s_cmax.empty:
        s_cmax.index.names = index_names
        dfs.append(s_cmax.rename('cmax_dur'))
        
    if not s_dcmax.empty:
        s_dcmax.index.names = index_names
        dfs.append(s_dcmax.rename('dcmax_dur'))
        
    if not s_cmin.empty:
        s_cmin.index.names = index_names
        dfs.append(s_cmin.rename('cmin_dur'))
        
    if not s_dcmin.empty:
        s_dcmin.index.names = index_names
        dfs.append(s_dcmin.rename('dcmin_dur'))
        
    if not dfs:
        return pd.DataFrame()
        
    merged = pd.concat(dfs, axis=1).fillna(0)
    
    # Ensure all duration columns exist
    target_dur_cols = ['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']
    merged = merged.reindex(columns=list(set(merged.columns) | set(target_dur_cols)), fill_value=0)
    
    # --- Calculations ---
    
    # Assign the clipped total time to the daily_dur column
    merged['daily_dur'] = total_day_mins 

    # Convert seconds to minutes for Duration columns
    merged[['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']] = merged[['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']] / 60.0
    
    # Calculate Percentages against the Total Day Time (mode-agnostic)
    if total_day_mins > 0:
        for col in target_dur_cols:
            merged[f'{col}_pct'] = (merged[col] / total_day_mins) * 100.0
    else:
        for col in target_dur_cols:
            merged[f'{col}_pct'] = 0.0
        
    # --- Formatting ---
    cols_to_round = ['daily_dur','cmax_dur', 'cmax_dur_pct', 'dcmax_dur', 'dcmax_dur_pct', 
                     'cmin_dur', 'cmin_dur_pct', 'dcmin_dur', 'dcmin_dur_pct']
    
    # Ensure all percentage columns exist
    for col in cols_to_round:
        if col not in merged.columns:
             merged[col] = 0.0
             
    merged[cols_to_round] = merged[cols_to_round].round(2)
    
    # Reset index and insert identifier columns
    merged = merged.reset_index()
    merged.insert(0, 'id', vid)
    merged.insert(0, 'date_val', date_val)
    
    # Reorder columns
    desired_order = ['date_val', 'id', 'tc_id', 'pack_id', 'daily_dur',
                     'cmax_dur', 'cmax_dur_pct', 'dcmax_dur', 'dcmax_dur_pct',
                     'cmin_dur', 'cmin_dur_pct', 'dcmin_dur', 'dcmin_dur_pct']
    
    return merged[desired_order]

In [ ]:
# df_sub = read_parquet_subset("df_with_state.parquet", datetime(2025, 9, 1), datetime(2025, 9, 2))
# df_sub.head(500).to_csv('hotspot_ip.csv')

In [ ]:
def run_multi_day_tc_hotspot_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    all_tables = []
    total_rows = 0

    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        for d in tqdm(range(num_days), desc="Processing Days", ncols=70):

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            df_sub["date_val"] = day_start.date()

            # per-vehicle
            for vid, g in df_sub.groupby("id"):
                res = compute_detailed_hotspot(g)
                if not res.empty:
                    res["id"] = vid
                    all_tables.append(res)

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    if not all_tables:
        return pd.DataFrame(), total_rows

    return pd.concat(all_tables, ignore_index=True), total_rows


start_date = datetime(2025, 9, 1)

tc_hotspot_df, total_rows = run_multi_day_tc_hotspot_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=start_date,
    num_days=75
)

print(tc_hotspot_df.head())
print("Total rows processed:", total_rows)

sort_cols = ['date_val', 'id', 'pack_id', 'tc_id']
tc_hotspot_df = tc_hotspot_df.sort_values(by=sort_cols, ascending=True).reset_index(drop=True)
tc_hotspot_df.to_csv('hotspot.csv')
display(tc_hotspot_df.head())

In [ ]:
pd.set_option('display.max_rows', 10)
tc_hotspot_df[tc_hotspot_df.daily_dur>1440].daily_dur.unique()

In [ ]:
pd.set_option('display.max_rows', None)
import pandas as pd

# Load the TC-level aggregated data (hotspot.csv)
df_hotspot = pd.read_csv('hotspot.csv')

# 1. Group by pack_id and sum all duration columns (NUMERATOR)
pack_agg = df_hotspot.groupby(['pack_id']).agg(
    total_cmax_dur=('cmax_dur', 'sum'),
    total_dcmax_dur=('dcmax_dur', 'sum'),
    total_cmin_dur=('cmin_dur', 'sum'),
    total_dcmin_dur=('dcmin_dur', 'sum'),
).reset_index()

# 2. Calculate the Grand Total for each duration column (DENOMINATOR)
grand_totals = pack_agg[['total_cmax_dur', 'total_dcmax_dur', 
                         'total_cmin_dur', 'total_dcmin_dur']].sum()

# 3. Calculate the percentage share for each pack
for col in ['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']:
    # Get the total duration for this specific metric
    total_duration = grand_totals[f'total_{col}']
    
    if total_duration > 0:
        # Calculate the pack's share of the total problem duration
        pack_agg[f'{col}_Share_Pct'] = (
            pack_agg[f'total_{col}'] / total_duration * 100
        ).round(2)
    else:
        pack_agg[f'{col}_Share_Pct'] = 0.0

# 4. Select the final output columns
hotspot_pack_level = pack_agg[['pack_id', 
                             'cmax_dur_Share_Pct', 'dcmax_dur_Share_Pct',
                             'cmin_dur_Share_Pct', 'dcmin_dur_Share_Pct']]

hotspot_pack_level.to_csv('hotspot_pack.csv')

In [ ]:
import pandas as pd
import numpy as np

def generate_high_delta_v_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Analyzes charging sessions for high DeltaV (> 20mV) and aggregates metrics 
    by SOC band across the entire fleet.
    """
    
    # Ensure necessary columns are correctly typed
    df['volt_delta_mv'] = pd.to_numeric(df['volt_delta_mv'], errors='coerce')
    df['total_battery_current'] = pd.to_numeric(df['total_battery_current'], errors='coerce')
    if 'dt_sec' in df.columns:
        df['dt_sec'] = pd.to_numeric(df['dt_sec'], errors='coerce')

    # --- 1. Filter Data ---
    df_charging = df[df['alt_mode'] == 'CHARGING'].copy()
    df_filtered = df_charging[df_charging['volt_delta_mv'] > 20].copy()
    
    # Prepare absolute current column
    df_filtered['abs_current'] = df_filtered['total_battery_current'].abs()

    # --- FIX 1: Update the column name in the final structure list ---
    final_report_cols_structure = [
        'total_ids', 'SOC band', '% Time', 
        'mv Imbalance (median)', 'mv Imbalance (P95)', # <--- FIXED: Now uses P95
        'Charging Current (Median)', 'Charging Current (Max)'
    ]
    # -----------------------------------------------------------------

    if df_filtered.empty:
        # Return an empty DataFrame with the correct structure
        return pd.DataFrame(columns=final_report_cols_structure)

    # --- 2. Calculate Total Vehicle Count ---
    total_unique_vehicles = df_filtered['id'].nunique()

    p95_func = lambda x: x.quantile(0.95)    

    # --- 3. Grouping and Aggregation ---
    grouped_by_soc = df_filtered.groupby('soc_band_bucket').agg(
        **{
            'mv Imbalance (median)': ('volt_delta_mv', 'median'),
            'mv Imbalance (P95)': ('volt_delta_mv', p95_func), # P95 is calculated here
            'Charging Current (Median)': ('abs_current', 'median'),
            'Charging Current (Max)': ('abs_current', 'max'),
            'Band Duration (sec)': ('dt_sec', 'sum')            
        }
    ).reset_index()

    # --- CALCULATE % TIME ---
    total_time_filtered = grouped_by_soc['Band Duration (sec)'].sum()

    if total_time_filtered > 0:
        grouped_by_soc['% Time'] = (
            grouped_by_soc['Band Duration (sec)'] / total_time_filtered * 100
        ).round(2)
    else:
        grouped_by_soc['% Time'] = 0.0
        
    # Drop the temporary duration column
    grouped_by_soc = grouped_by_soc.drop(columns=['Band Duration (sec)'])

    # --- 4. Final Formatting ---
    final_report = grouped_by_soc.rename(columns={'soc_band_bucket': 'SOC band'})
    
    # Insert the 'Vehicle ID' column and assign the total count
    final_report.insert(0, 'total_ids', total_unique_vehicles)

    # --- FIX 2: Update rounding logic to include P95 ---
    for col in final_report.columns:
        # Check for 'median', 'P95', 'Max', or 'Time' to round all metrics
        if 'median' in col or 'P95' in col or 'Max' in col or 'Time' in col:
            final_report[col] = final_report[col].round(2)
    # ----------------------------------------------------
            
    # Ensure final column order is correct (now that P95 is in the structure list)
    final_report = final_report[[col for col in final_report_cols_structure if col in final_report.columns]]

    return final_report

In [ ]:
def run_multi_day_deltav_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    """
    Reads multi-day Parquet data and computes the fleet-wide high DeltaV report, 
    using a progress bar for the outer loop.
    """
    raw_data_for_deltav = []
    total_rows = 0

    # Logging setup (remains the same to avoid NameError)
    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        # --- CHANGE HERE: Wrap range(num_days) with tqdm ---
        # NOTE: You need to have 'from tqdm import tqdm' imported in your environment
        # Use a descriptive label like "DeltaV Analysis Days"
        for d in tqdm(range(num_days), desc="DeltaV Analysis Days", ncols=70): 
        # --------------------------------------------------

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            # --- Replace with your actual Parquet reading function ---
            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            
            # --- COLLECT RAW DATA ---
            cols_needed = ['id', 'dt_sec','alt_mode', 'volt_delta_mv', 'total_battery_current', 'soc_band_bucket']
            cols_present = [col for col in cols_needed if col in df_sub.columns]
            
            raw_data_for_deltav.append(df_sub[cols_present].copy())

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    # -------------------------------------------------------------
    # RUN FLEET-WIDE DELTAV ANALYSIS (ONCE)
    # -------------------------------------------------------------
    if raw_data_for_deltav:
        full_raw_data_df = pd.concat(raw_data_for_deltav, ignore_index=True)
        delta_v_report_df = generate_high_delta_v_report(full_raw_data_df)
    else:
        delta_v_report_df = generate_high_delta_v_report(pd.DataFrame())

    return delta_v_report_df, total_rows


final_report, total_rows = run_multi_day_deltav_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=datetime(2025, 9, 1),
    num_days=75
)

In [ ]:
final_report
final_report.to_csv('soc_band_mv_current_analysis.csv')

In [ ]:
import pandas as pd
import numpy as np

def generate_thermal_voltage_hotspot_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates a report showing Thermocouple (TC) Hotspot exposure duration 
    (Max Temp/Min Temp) during high voltage imbalance (volt_delta_mv > 5).
    
    Required columns: dt_sec, volt_delta_mv, batt_maxtemp_tc, batt_mintemp_tc.
    """
    
    # --- 1. Data Type Conversion and Preparation ---
    df['volt_delta_mv'] = pd.to_numeric(df['volt_delta_mv'], errors='coerce')
    
    # Check for duration column and convert to minutes
    if 'dt_sec' not in df.columns:
        print("Error: 'dt_sec' column missing for duration calculation.")
        return pd.DataFrame() 

    df['dt_min'] = pd.to_numeric(df['dt_sec'], errors='coerce') / 60.0

    # --- 2. Filtering ---
    # Apply condition: volt_delta_mv > 5 (High Voltage Imbalance)
    df_filtered = df[df['volt_delta_mv'] > 5].copy()
    
    final_report_cols_structure = [
        'Thermocouple ID', 'Max Temp Duration (min)', 'Min Temp Duration (min)'
    ]

    if df_filtered.empty:
        return pd.DataFrame(columns=final_report_cols_structure)

    # --- 3. Aggregation for MAX Temperature Duration ---
    # Group by the TC ID that was reporting the MAX temp and sum the duration.
    max_temp_df = df_filtered.groupby('batt_maxtemp_tc')['dt_min'].sum().reset_index()
    max_temp_df = max_temp_df.rename(columns={
        'batt_maxtemp_tc': 'Thermocouple ID',
        'dt_min': 'Max Temp Duration (min)'
    })

    # --- 4. Aggregation for MIN Temperature Duration ---
    # Group by the TC ID that was reporting the MIN temp and sum the duration.
    min_temp_df = df_filtered.groupby('batt_mintemp_tc')['dt_min'].sum().reset_index()
    min_temp_df = min_temp_df.rename(columns={
        'batt_mintemp_tc': 'Thermocouple ID',
        'dt_min': 'Min Temp Duration (min)'
    })

    # --- 5. Merge and Cleanup ---
    # Perform an Outer Join to combine both results, keeping all TCs
    final_report = pd.merge(
        max_temp_df, 
        min_temp_df, 
        on='Thermocouple ID', 
        how='outer'
    )

    # Fill NaNs created by the outer merge with 0
    final_report = final_report.fillna(0)

    # Round all duration metrics
    for col in final_report.columns:
        if 'Duration' in col:
            final_report[col] = final_report[col].round(2)
            
    # Filter out TCs with zero total exposure
    final_report = final_report[
        (final_report['Max Temp Duration (min)'] > 0) | 
        (final_report['Min Temp Duration (min)'] > 0)
    ]
    
    # Ensure ID column is integer (since TC IDs are usually integers)
    final_report['Thermocouple ID'] = final_report['Thermocouple ID'].astype(pd.Int64Dtype())

    return final_report


def run_multi_day_deltav_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    """
    Reads multi-day Parquet data and computes the fleet-wide high DeltaV report, 
    using a progress bar for the outer loop.
    """
    raw_data = []
    total_rows = 0

    # Logging setup (remains the same to avoid NameError)
    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        # --- CHANGE HERE: Wrap range(num_days) with tqdm ---
        # NOTE: You need to have 'from tqdm import tqdm' imported in your environment
        # Use a descriptive label like "DeltaV Analysis Days"
        for d in tqdm(range(num_days), desc="ThermalV Analysis Days", ncols=70): 
        # --------------------------------------------------

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            # --- Replace with your actual Parquet reading function ---
            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            
            # --- COLLECT RAW DATA ---
            cols_needed = ['batt_maxtemp_tc','batt_mintemp_tc', 'dt_sec', 'volt_delta_mv']
            cols_present = [col for col in cols_needed if col in df_sub.columns]
            
            raw_data.append(df_sub[cols_present].copy())

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    # -------------------------------------------------------------
    # RUN FLEET-WIDE DELTAV ANALYSIS (ONCE)
    # -------------------------------------------------------------
    if raw_data:
        full_raw_data_df = pd.concat(raw_data, ignore_index=True)
        delta_v_report_df = generate_thermal_voltage_hotspot_report(full_raw_data_df)
    else:
        delta_v_report_df = generate_thermal_voltage_hotspot_report(pd.DataFrame())

    return delta_v_report_df, total_rows


thermal_v_tc_hotspot_report, total_rows = run_multi_day_deltav_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=datetime(2025, 9, 1),
    num_days=75
)

display(thermal_v_tc_hotspot_report.head())


In [ ]:
thermal_v_tc_hotspot_report.to_csv('hotspot_tc.csv')

In [ ]:
import pandas as pd
import numpy as np

def generate_thermal_voltage_pack_hotspot_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates a report showing Pack Hotspot exposure duration 
    (Max Temp/Min Temp) during high voltage imbalance (volt_delta_mv > 5).
    
    Required columns: dt_sec, volt_delta_mv, batt_maxtemp_tc, batt_mintemp_tc.
    """
    
    # --- 1. Data Type Conversion and Preparation ---
    df['volt_delta_mv'] = pd.to_numeric(df['volt_delta_mv'], errors='coerce')
    
    # Check for duration column and convert to minutes
    if 'dt_sec' not in df.columns:
        print("Error: 'dt_sec' column missing for duration calculation.")
        return pd.DataFrame() 

    df['dt_min'] = pd.to_numeric(df['dt_sec'], errors='coerce') / 60.0

    # --- 2. Filtering ---
    # Apply condition: volt_delta_mv > 5 (High Voltage Imbalance)
    df_filtered = df[df['volt_delta_mv'] > 5].copy()
    
    final_report_cols_structure = [
        'Pack ID', 'Max Temp Duration (min)', 'Min Temp Duration (min)'
    ]

    if df_filtered.empty:
        return pd.DataFrame(columns=final_report_cols_structure)

    # --- 3. Aggregation for MAX Temperature Duration ---
    # Group by the TC ID that was reporting the MAX temp and sum the duration.
    max_temp_df = df_filtered.groupby('pack_id_max')['dt_min'].sum().reset_index()
    max_temp_df = max_temp_df.rename(columns={
        'pack_id_max': 'Pack ID',
        'dt_min': 'Max Temp Duration (min)'
    })

    # --- 4. Aggregation for MIN Temperature Duration ---
    # Group by the TC ID that was reporting the MIN temp and sum the duration.
    min_temp_df = df_filtered.groupby('pack_id_min')['dt_min'].sum().reset_index()
    min_temp_df = min_temp_df.rename(columns={
        'pack_id_min': 'Pack ID',
        'dt_min': 'Min Temp Duration (min)'
    })

    # --- 5. Merge and Cleanup ---
    # Perform an Outer Join to combine both results, keeping all TCs
    final_report = pd.merge(
        max_temp_df, 
        min_temp_df, 
        on='Pack ID', 
        how='outer'
    )

    # Fill NaNs created by the outer merge with 0
    final_report = final_report.fillna(0)

    # Round all duration metrics
    for col in final_report.columns:
        if 'Duration' in col:
            final_report[col] = final_report[col].round(2)
            
    # Filter out TCs with zero total exposure
    final_report = final_report[
        (final_report['Max Temp Duration (min)'] > 0) | 
        (final_report['Min Temp Duration (min)'] > 0)
    ]
    
    # Ensure ID column is integer (since TC IDs are usually integers)
    final_report['Pack ID'] = final_report['Pack ID'].astype(pd.Int64Dtype())

    return final_report


def run_multi_day_deltav_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    """
    Reads multi-day Parquet data and computes the fleet-wide high DeltaV report, 
    using a progress bar for the outer loop.
    """
    raw_data = []
    total_rows = 0

    # Logging setup (remains the same to avoid NameError)
    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        # --- CHANGE HERE: Wrap range(num_days) with tqdm ---
        # NOTE: You need to have 'from tqdm import tqdm' imported in your environment
        # Use a descriptive label like "DeltaV Analysis Days"
        for d in tqdm(range(num_days), desc="ThermalV Analysis Days", ncols=70): 
        # --------------------------------------------------

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            # --- Replace with your actual Parquet reading function ---
            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            
            # --- COLLECT RAW DATA ---
            cols_needed = ['pack_id_max','pack_id_min', 'dt_sec', 'volt_delta_mv']
            cols_present = [col for col in cols_needed if col in df_sub.columns]
            
            raw_data.append(df_sub[cols_present].copy())

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    # -------------------------------------------------------------
    # RUN FLEET-WIDE DELTAV ANALYSIS (ONCE)
    # -------------------------------------------------------------
    if raw_data:
        full_raw_data_df = pd.concat(raw_data, ignore_index=True)
        delta_v_report_df = generate_thermal_voltage_pack_hotspot_report(full_raw_data_df)
    else:
        delta_v_report_df = generate_thermal_voltage_pack_hotspot_report(pd.DataFrame())

    return delta_v_report_df, total_rows


thermal_v_pack_hotspot_report, total_rows = run_multi_day_deltav_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=datetime(2025, 9, 1),
    num_days=75
)

display(thermal_v_pack_hotspot_report)

In [ ]:
thermal_v_pack_hotspot_report.to_csv('hotspot_pack.csv')

In [ ]:
# pd.set_option('display.max_rows', 100)
# tc_hotspot_df.groupby(['date_val','id','daily_dur','pack_id'])[['cmax_dur_pct','dcmax_dur_pct','cmin_dur_pct','dcmin_dur_pct']].sum().round(2).head(7)